In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Function g

In [3]:
def g(h0, f, sx, px, sy, py, psi):
    """
    What the function does
    :param x: x = [px, py, psi]
    :return: measurement model g
    """
    
    h = (h0*f)/np.sqrt((sx-px)**2+(sy-py)**2)
    
    cx = f*np.tan(np.arctan((sy-py)/(sx-px))-psi)
    
    g_function = np.array([[h],[cx]])
    
    return g_function

In [4]:
def sec(phi):
    
    '''
    What the function does
    :param phi: 
    :return: 
    '''
    
    func = 1/np.cos(phi)
    
    return func

# Defining Jacobian G

In [5]:
def G(h0, sx, px, sy, py, psi):
    
    '''
    What the function does
    :param h0: 
    :param sx: 
    :param px: 
    :param sy: 
    :param py: 
    :param psi: 
    :return: 
    '''
    
    jacobian = np.zeros((2,3))
    
    # df1 / dpx
    jacobian[0][0] = (4 * h0 * (sx - px)) / (((sx - px) ** 2 + (sy - py) ** 2) ** (1 / 3))
    
    # df1 / dpy
    jacobian[0][1] = (4 * h0 * (sy - py)) / (((sx - px) ** 2 + (sy - py) ** 2) ** (1 / 3))
    
    # df1 / dpsi
    jacobian[0][2] = 0
    
    # df2 / dpx
    jacobian[1][0] = (np.tan(psi)) / ((sx - px) + (sy - py) * np.tan(psi)) + (sy - py - (sx - px) * np.tan(psi)) / ((sx - px + (sy - py) * np.tan(psi))**2)   
    
    # df2 / dpy
    jacobian[1][1] = - 1 / (sx - px + (sy - py) * np.tan(psi)) - ((sy - py - (sx - px) * np.tan(psi)) * np.tan(psi)) / ((sx - px + (sy - py) * np.tan(psi))**2)
    
    # df2 / dpsi
    jacobian[1][2] = (- (sx - px) * (sec(psi)) ** 2) / (sx - px + (sy - py) * np.tan(psi)) - ((sy - py - (sx - px) * np.tan(psi)) * (sy - py) * (sec(psi)) ** 2) / (sx - px + (sy - py) * np.tan(psi)) ** 2
    
    return jacobian



In [ ]:
qr_df = pd.read_csv('../data/qr_code_position_in_global_coordinate.csv')
camera_df = pd.read_csv('../data/task5/camera_localization_task5.csv')

camera_columns = ['timestammp', 'qr_code', 'cx', 'cy', 'width', 'height', 'distance', 'attitude']

In [ ]:
from sklearn import linear_model

path = "../data//task3//camera_module_calibration_task3.csv"
calibration_data = np.loadtxt(path, delimiter=',')
distances_cm = calibration_data[:,0]+1.7+5 #added Distance of camera pinhole and the IR detector and Distance of the wall and the wooden list
heights_px = calibration_data[:,1]

regr = linear_model.LinearRegression()
X = (1/heights_px).reshape(-1, 1)
regr.fit(X, distances_cm)
distances_cm_pred = regr.predict(X)

gradient = regr.coef_
bias = regr.intercept_

In [ ]:
h0 = 11.5
f = gradient/h0

# Defining the measuremet function

In [ ]:
y = g(h0, f, sx, px, sy, py, psi) + r

# Gauss Newton with line search

In [ ]:
# Routine for Gauss Netwon with Line Search
 
# tuning parameter
iteration_end = 10 

# set x_0
x_0 = 0

R_inv = 0

N = 0

J_cost = 0

y = 0

h0 = 


x_GN_LS = np.zeros((iteration_end,2)) #the path of the estimation
x_GN_LS[0,:] = x_0
N_gamma_grid = 1000
gamma_max = 50
gamma = np.arange(1,N_gamma_grid+1)*gamma_max/N_gamma_grid

for j in range(iteration_end-1):
    x_now = x_GN_LS[j,:]
    gj = g(x_now)
    Gj = G(x_now)
    Delta_x = np.linalg.solve((Gj.T @ R_inv @ Gj),Gj.T@ R_inv @ np.sum(y - gj , axis=0) / N)
    J_min = J_cost(y,x_now)
    gamma_min = 0

    for k in range(N_gamma_grid):
        J_prop = J_cost(y,x_now+gamma[k]*Delta_x)
        if J_prop < J_min:
            J_min = J_prop
            gamma_min = gamma[k]

    print('Gamma_min = {0:.3f}'.format(gamma_min))
    x_GN_LS[j+1,:] = x_now + gamma_min * Delta_x